In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import joblib
import requests
import eventlet
import time
import socketio
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBRegressor
from flask import Flask, request, jsonify
from binance.client import Client
from apscheduler.schedulers.background import BackgroundScheduler
from datetime import date
from flask_socketio import SocketIO

In [ ]:
app = Flask(__name__)
socketio = SocketIO(app, cors_allowed_origins="*")

## **Gathering Data**

In [ ]:
btc_ticker = yf.Ticker("SUI-USD")
btc = btc_ticker.history(period="max")

In [ ]:
btc

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2022-03-18 00:00:00+00:00,0.009705,0.011553,0.009705,0.009937,302479,0.0,0.0
2022-03-19 00:00:00+00:00,0.009937,0.009999,0.008678,0.008748,117482,0.0,0.0
2022-03-20 00:00:00+00:00,0.008748,0.008776,0.008261,0.008342,55250,0.0,0.0
2022-03-21 00:00:00+00:00,0.008342,0.009224,0.008144,0.008964,41834,0.0,0.0
2022-03-22 00:00:00+00:00,0.008964,0.009223,0.008800,0.008853,39925,0.0,0.0
...,...,...,...,...,...,...,...
2024-05-31 00:00:00+00:00,0.000300,0.000300,0.000300,0.000300,0,0.0,0.0
2024-06-01 00:00:00+00:00,0.000300,0.000300,0.000300,0.000300,0,0.0,0.0
2024-06-02 00:00:00+00:00,0.000300,0.000300,0.000300,0.000300,0,0.0,0.0


## **Preparing Data**

In [ ]:
def calculate_rsi(data, window=14):
    delta = data["Close"].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

In [ ]:
def prepare_btc(btc):
  btc.index = pd.to_datetime(btc.index)
  btc.index = btc.index.tz_convert(None)

  del btc["Dividends"]
  del btc["Stock Splits"]

  btc["SMA50"] = btc["Close"].rolling(window=50).mean()
  btc["SMA100"] = btc["Close"].rolling(window=100).mean()
  btc["SMA200"] = btc["Close"].rolling(window=200).mean()
  btc['Volatility'] = btc['Close'].pct_change().rolling(window=30).std()

  btc["Future_Close"] = btc["Close"].shift(-7)

  btc["MACD"] = btc["Close"].ewm(span=12, adjust=False).mean() - btc["Close"].ewm(span=26, adjust=False).mean()
  btc["Signal"] = btc["MACD"].ewm(span=9, adjust=False).mean()

  btc["RSI"] = calculate_rsi(btc)

  btc.dropna(inplace=True)

  return btc

In [ ]:
btc = prepare_btc(btc)

In [ ]:
btc

,Open,High,Low,Close,Volume,SMA50,SMA100,SMA200,Volatility,Future_Close,MACD,Signal,RSI
Date,,,,,,,,,,,,,
2022-10-03,0.004044,0.004103,0.004024,0.004066,1407,0.003857,0.003530,0.004483,0.025709,0.003942,0.000059,0.000026,77.718369
2022-10-04,0.004065,0.004233,0.004055,0.004205,504,0.003854,0.003547,0.004455,0.026312,0.003896,0.000076,0.000036,94.108015
2022-10-05,0.004204,0.004224,0.004113,0.004183,422,0.003851,0.003564,0.004432,0.026317,0.003837,0.000088,0.000046,91.853015
2022-10-06,0.004176,0.004256,0.004005,0.004069,1009,0.003849,0.003580,0.004410,0.025527,0.003814,0.000087,0.000054,71.354160
2022-10-07,0.004069,0.004180,0.004025,0.004155,1413,0.003851,0.003599,0.004386,0.024407,0.004258,0.000091,0.000062,74.924012
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-24,0.000292,0.000464,0.000292,0.000464,7,0.000401,0.000309,0.001567,0.485134,0.000300,-0.000018,-0.000017,52.702704
2024-05-25,0.000464,0.000690,0.000462,0.000462,17,0.000402,0.000313,0.001550,0.483354,0.000300,-0.000009,-0.000015,50.615114
2024-05-26,0.000462,0.000462,0.000449,0.000452,4,0.000403,0.000317,0.001533,0.478316,0.000300,-0.000003,-0.000013,46.341463


In [ ]:
features = ['SMA50', 'SMA100', 'SMA200', 'Volatility', 'MACD', 'Signal', 'RSI']
target = 'Future_Close'

In [ ]:
scaler = MinMaxScaler()
btc[features] = scaler.fit_transform(btc[features])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(btc[features], btc[target], test_size = 0.2)

In [ ]:
model = RandomForestRegressor()

In [ ]:
model.fit(X_train, y_train)
joblib.dump(model, "btc_model.pkl")
prediction = model.predict(X_test)

In [ ]:
mse = np.sqrt(mean_squared_error(y_test, prediction))

In [ ]:
mse

0.0003408617101575516

In [ ]:
future_features = btc[features].iloc[-7:]
future_predictions = model.predict(future_features)

print("Predicted Prices for Next 7 Days:", future_predictions)

Predicted Prices for Next 7 Days: [96642.17960938 96620.59671875 97137.59984375 96420.74429687
 97788.11414062 97292.65859375 98070.22695313]


In [ ]:
r2 = r2_score(y_test, prediction)

r2

0.9967892245296144

## **Using XGBoost**

In [ ]:
model = XGBRegressor(n_estimator=200, learning_rate=0.05)

In [ ]:
model.fit(X_train, y_train)
prediction = model.predict(X_test)

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [20:18:09] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "n_estimator" } are not used.

  warnings.warn(smsg, UserWarning)


In [ ]:
mse = np.sqrt(mean_squared_error(y_test, prediction))

mse

1422.9409051439366

## **DCA Strategy**

In [ ]:
investment_amount = 100
investment_threshold = np.percentile(future_predictions, 30)

In [ ]:
def dca_strategy(prediction, threshold):
    investment_schedule = []
    for price in prediction:
        if price < threshold:
            investment_schedule.append(investment_amount * 1.5)
        else:
            investment_schedule.append(investment_amount)
    return investment_schedule

In [ ]:
dca_investments = dca_strategy(future_predictions, investment_threshold)
print("Investment Schedule:", dca_investments)

Investment Schedule: [100, 150.0, 100, 150.0, 100, 100, 100]


## **Live Data**

In [ ]:
def train_model():
  btc_ticker = yf.Ticker("SOL-USD")
  new_btc = btc_ticker.history(date=date.today().strftime("%Y-%m-%d"))
  new_btc = prepare_btc(new_btc)

  model = joblib.load("btc_model.pkl")

  model.fit(new_btc[features], new_btc[target])
  joblib.dump(model, "btc_model.pkl")

In [ ]:
scheduler = BackgroundScheduler()
scheduler.add_job(train_model, 'interval', hours=24)
scheduler.start()

In [ ]:
@app.route('/predict', methods=['GET'])
def predict():
  dca_investments = dca_strategy(future_predictions, investment_threshold)
  return jsonify({
        "predicted_prices": future_predictions.tolist(),
        "investment_schedule": dca_strategy(future_predictions, investment_threshold)
        })

In [ ]:
if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


In [ ]:
def generate_live_predictions():
    while True:
        # Simulate real-time features (Replace this with live data fetching)
        live_features = np.random.rand(1, 7)  # Assuming model expects 7 features

        # Make prediction
        predicted_price = model.predict(live_features)[0]

        # Emit the prediction to connected clients
        socketio.emit("price_update", {"predicted_price": predicted_price})

        print(f"Sent Predicted Price: {predicted_price}")

        time.sleep(10)  # Send updates every 10 seconds

In [ ]:
@socketio.on("connect")
def handle_connect():
    print("Client connected!")
    socketio.start_background_task(generate_live_predictions)

In [ ]:
@app.route("/")
def index():
    return jsonify({"message": "WebSocket server is running!"})

In [ ]:
if __name__ == "__main__":
    socketio.run(app, host="0.0.0.0", port=5000, debug=True)

INFO:werkzeug: * Restarting with stat


In [ ]:
sio = socketio.Client()

@sio.on("price_update")
def on_message(data):
    print("📈 Live Predicted Price:", data["predicted_price"])

sio.connect("http://localhost:5000")
sio.wait()

ConnectionError: HTTPConnectionPool(host='localhost', port=5000): Max retries exceeded with url: /socket.io/?transport=polling&EIO=4&t=1739564502.3068871 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x790678c96610>: Failed to establish a new connection: [Errno 111] Connection refused'))